In [10]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [11]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

        
    return movie

In [12]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)
import os

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(f'{kaggle_file}', low_memory=False)
    ratings = pd.read_csv(f'{ratings_file}')    

    # Open and read the Wikipedia data JSON file.
    with open(f'{wiki_file}', mode='r') as file:
        wiki_movies_raw = json.load(file)
#--------
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw if 'No. of episodes' not in movie]
#--------
    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]
#--------
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)
#--------
    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})') # captures the 7 digits in imdb link https://www.imdb.com/title/tt0098987/
        wiki_movies_df.drop_duplicates(subset="imdb_id", inplace=True)
    except (ValueError):
        print('Could not find IMDb ID.')
#--------
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df)* 0.9] #why *.9 can i convert this into a df
    wiki_movies_df = wiki_movies_df[wiki_columns_keep]
#--------
    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
#--------    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
#--------
    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
#--------
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
#--------
    # 12. Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
            if type(s) != str:
                return np.nan

        # if input is of the form $###.# million
            if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
                s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
                value = float(s) * 10**6

            # return value
                return value

        # if input is of the form $###.# billion
            elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
                s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
                value = float(s) * 10**9

            # return value
                return value

        # if input is of the form $###,###,###
            elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
                s = re.sub('\$|,','', s)

            # convert to float
                value = float(s)

            # return value
                return value

        # otherwise, return NaN
            else:
                return np.nan

#--------    
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True) # will remove range like $50-75 million
    
    # extract form_one and form_two and parse
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    
    # drop old column
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
#--------    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    # Create a budget variable
    budget = wiki_movies_df['Budget'].dropna()

    # Convert any lists to strings:
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
   
    # Remove the range references
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    
    # extract the form_one and form_two matches from the budget column and parse
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
#--------
    
    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

    # create a list of formats that match the variables for release date
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    
    # Extract the dates
    release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
    
    # Parse the release date
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

    # Drop Release date column: wiki_movies_df.drop('Release date', axis=1, inplace=True)
#--------
    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    # make a variable that holds the non-null values of running time in the DataFrame, converting lists to strings:
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    running_time[running_time.str.contains(r'\d*\s*m', flags=re.IGNORECASE) != True]
    
    # extract the digits for running time
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    
    # convert to numeric values
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    
    # save to the dataframe
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    
    # drop old runtime column:
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
#--------
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 

In [13]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = os.path.join('Resources') #don't really need this when using os on the variables
# Wikipedia data
wiki_file = os.path.join('Resources', "wikipedia-movies.json")
# Kaggle metadata
kaggle_file = os.path.join('Resources', 'movies_metadata.csv')
# MovieLens rating data.
ratings_file = os.path.join('Resources', 'ratings.csv')

In [14]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

In [15]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [16]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.sample(3)

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
35,https://en.wikipedia.org/wiki/The_China_Lake_M...,1990.0,https://www.imdb.com/title/tt0099259/,The China Lake Murders,NaN,"[Isaac Allan, Tom Skerritt, Michael Parks]",Geoffrey Schaaf,"[January 31, 1990, (, 1990-01-31, )]",United States,NaN,...,Sidney Wolinsky,Dana Kaproff,"[William Beaudine Jr., Beth Tate (co-producer)]","[MCA Television Entertainment, Papazian-Hirsch...",N.D. Schreiner,tt0099259,NaN,NaN,1990-01-31,97.0
3592,https://en.wikipedia.org/wiki/Before_Sunset,2004.0,https://www.imdb.com/title/tt0381681/,Before Sunset,"[Characters, by Richard Linklater, and Kim Kri...","[Ethan Hawke, Julie Delpy]",Lee Daniel,"[February 10, 2004, (, 2004-02-10, ), (, BIFF,...",United States,"[English, French]",...,Sandra Adair,Cliff Eidelman,"[Richard Linklater, Anne Walker-McBay]",Castle Rock Entertainment,"[Richard Linklater, Kim Krizan]",tt0381681,16000000.0,2700000.0,2004-02-10,80.0
3930,https://en.wikipedia.org/wiki/Just_Like_Heaven...,2005.0,https://www.imdb.com/title/tt0425123/,Just Like Heaven,"[If Only It Were True, by, Marc Levy]","[Reese Witherspoon, Mark Ruffalo]",Daryn Okada,"[September 16, 2005, (, 2005-09-16, )]",United States,English,...,Bruce Green,Rolfe Kent,"[Walter F. Parkes, Laurie MacDonald]",NaN,"[Peter Tolan, Leslie Dixon]",tt0425123,102800000.0,58000000.0,2005-09-16,95.0


In [17]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Country',
 'Language',
 'Budget',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id',
 'box_office',
 'budget',
 'release_date',
 'running_time']